In [1]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import os
from sklearn.datasets import load_files
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import mglearn

In [3]:
df = pd.read_csv('metadata.csv') 
df

,doc_id,publication_time,bp,doc_type,minister
0,19700101_ACO_1646_3420566,1970-01-01,[10],ACO,joaquim barbosa
1,19700101_AC_2287_3093972,1970-01-01,[4],AC,ellen gracie
2,19700101_AC_2459_3245342,1970-01-01,[3],AC,cármen lúcia
3,19700101_AC_2459_3258903,1970-01-01,[3],AC,cármen lúcia
4,19700101_AC_2507_3270033,1970-01-01,[17],AC,joaquim barbosa
...,...,...,...,...,...
29738,20181219_Rcl_31921_15339279527,2018-12-19,"[10, 37]",Rcl,gilmar mendes
29739,20181219_Rcl_32262_15339279531,2018-12-19,[10],Rcl,gilmar mendes
29740,20181219_Rcl_32695_15339271501,2018-12-19,[14],Rcl,rosa weber
29741,20181219_Rcl_32813_15339276760,2018-12-19,[10],Rcl,edson fachin


In [7]:
ids = list(df['doc_id'])
bp_label = list(df['bp'])
bp_set = list(set(df['bp']))
bp_set_single_bp = [element for element in bp_set if len(element)<5]
bp_set_single_bp

['[33]', '[3]', '[37]', '[4]', '[14]', '[17]', '[26]', '[20]', '[10]', '[11]']

In [16]:
if os.path.isdir('allbp') == False:
    os.mkdir('allbp') 
    
if os.path.isdir('allbp/3') == False:
    os.mkdir('allbp/3') 
    
if os.path.isdir('allbp/33') == False:
    os.mkdir('allbp/33') 

if os.path.isdir('allbp/37') == False:
    os.mkdir('allbp/37')

if os.path.isdir('allbp/4') == False:
    os.mkdir('allbp/4') 
    
if os.path.isdir('allbp/14') == False:
    os.mkdir('allbp/14') 
    
if os.path.isdir('allbp/17') == False:
    os.mkdir('allbp/17') 
    
if os.path.isdir('allbp/26') == False:
    os.mkdir('allbp/26') 

if os.path.isdir('allbp/20') == False:
    os.mkdir('allbp/20') 
    
if os.path.isdir('allbp/10') == False:
    os.mkdir('allbp/10') 

if os.path.isdir('allbp/11') == False:
    os.mkdir('allbp/11') 

In [19]:
for index in range(len(ids)):
    string = 'decisions_allbps/decisions/'+ids[index]+'.txt'
    if bp_label[index] == '[3]':
        shutil.move(string, 'allbp/3')
    elif bp_label[index] == '[33]':
        shutil.move(string, 'allbp/33')
    elif bp_label[index] == '[37]':
        shutil.move(string, 'allbp/37')
    elif bp_label[index] == '[4]':
        shutil.move(string, 'allbp/4')
    elif bp_label[index] == '[14]':
        shutil.move(string, 'allbp/14')
    elif bp_label[index] == '[17]':
        shutil.move(string, 'allbp/17')
    elif bp_label[index] == '[26]':
        shutil.move(string, 'allbp/26')
    elif bp_label[index] == '[20]':
        shutil.move(string, 'allbp/20')
    elif bp_label[index] == '[10]':
        shutil.move(string, 'allbp/10')
    elif bp_label[index] == '[11]':
        shutil.move(string, 'allbp/11')

In [22]:
bp = load_files("allbp")
bp_train, label_train = bp.data, bp.target
print("type of text_train: {}".format(type(bp_train)))
print("length of text_train: {}".format(len(bp_train)))
print("text_train[1]:\n{}".format(bp_train[1].decode("utf-8")))

type of text_train: <class 'list'>
length of text_train: 28534
text_train[1]:
EMB.DECL. NO MANDADO DE INJUNÇÃO 4.391 DISTRITO FEDERAL

RELATOR : MIN. RICARDO LEWANDOWSKI
EMBTE.(S) :ESTADO DE MATO GROSSO DO SUL 
PROC.(A/S)(ES) :ULISSES SCHWARZ VIANA 
EMBDO.(A/S) :PAULO ROBERTO STANGARLIN FERNANDES 
ADV.(A/S) :JACQUES CARDOSO DA CRUZ 
INTDO.(A/S) :PRESIDENTE DA REPÚBLICA 
ADV.(A/S) :ADVOGADO-GERAL DA UNIÃO 

Trata-se de embargos de declaração opostos pelo Estado de Mato 
Grosso do Sul contra a decisão, proferida em 13/3/2012, que, considerada 
a falta do diploma regulamentador a que se refere o art. 40, § 4º, III, da 
Constituição Federal, concedeu parcialmente a ordem para que o pleito 
de aposentadoria especial da servidora pública impetrante pudesse ser 
analisado pela autoridade administrativa, a quem compete a verificação 
do preenchimento ou não dos requisitos legais, em especial os do art. 57 
da Lei 8.213/1991.

É o breve relatório. Decido.

Bem examinados os autos, verifico que 

In [26]:
label_set = list(set(label_train))
for element in label_set:
    print(element, '---->', list(label_train).count(element))

0 ----> 14429
1 ----> 788
2 ----> 1539
3 ----> 840
4 ----> 1654
5 ----> 766
6 ----> 1052
7 ----> 1038
8 ----> 3089
9 ----> 3339


In [27]:


#produces the balanced set
bp_37 = [bp_train[index] for index in range(len(label_train)) if label_train[index]==1]
bp_train_bal = bp_10_bal+bp_37
bp_train_bal = np.array(bp_train_bal)
label_train_bal = list(np.zeros(len(bp_10_bal)))+list(np.ones(len(bp_37)))
label_train_bal = np.array(label_train_bal)
print(len(bp_train_bal),'    ', len(label_train_bal))

In [28]:
#take only 760 elements of each bp
bp_train_bal = []
label_train_bal = []

for bp in label_set:
    bp_only = [bp_train[index] for index in range(len(label_train)) if label_train[index]==bp]
    random_indexes = np.random.permutation(len(bp_only))
    bp_only = [bp_only[index] for index in random_indexes[:760]]
    bp_train_bal += bp_only
    label_train_bal += [bp for i in range(len(bp_only))]

In [31]:
def masker(dataset):
    '''Substitues all integers in a text dataset by an empty string
    Input: dataset made of texts
    Output: the same dataset made of texts, only with masked integers'''
    masked_set = []
    for index in range(len(dataset)):
        new_string = ''
        for word in range(len(dataset[index])):
            if dataset[index][word].isnumeric()==False:
                new_string += dataset[index][word]
            else:
                new_string += ''
        masked_set.append(new_string)
    return masked_set

In [32]:
uft_dataset = [bp_train_bal[index].decode("utf-8") for index in range(len(bp_train_bal))]

In [33]:
masked_dataset = masker(uft_dataset)
masked_dataset[0]

'RECURSO EXTRAORDINÁRIO COM AGRAVO . ALAGOAS\n\nRELATOR : MIN. LUIZ FUX\nRECTE.(S) :UNIÃO \nPROC.(A/S)(ES) :ADVOGADO-GERAL DA UNIÃO \nRECDO.(A/S) :FEDERACAO NACIONAL DOS POLICIAIS FEDERAIS \nADV.(A/S) :RICARDO LOBO CAVALCANTI DE ALBUQUERQUE \n\nRECURSO EXTRAORDINÁRIO COM \nAGRAVO. ADMINISTRATIVO. \nEMBARGOS À EXECUÇÃO DE \nSENTENÇA CONTRA A FAZENDA \nPÚBLICA. VIOLAÇÃO DO ART.  DO \nCPC. ALEGAÇÃO DE OFENSA AOS \nPRINCÍPIOS DO CONTRADITÓRIO, DA \nAMPLA DEFESA, DO DEVIDO \nPROCESSO LEGAL E DE AUSÊNCIA DE \nFUNDAMENTAÇÃO. DECISÃO DE \nADMISSIBILIDADE DO RECURSO \nEXTRAORDINÁRIO. APLICAÇÃO DE \nPRECEDENTES DESTA CORTE QUE \nRECONHECERAM A INEXISTÊNCIA DE \nREPERCUSSÃO GERAL DAS \nMATÉRIAS. INTERPOSIÇÃO DE \nAGRAVO. NÃO CABIMENTO. \nVIOLAÇÃO À CLÁUSULA DE RESERVA \nDE PLENÁRIO. INOCORRÊNCIA. \nAGRAVO NÃO CONHECIDO NO QUE \nSE REFERE À MATÉRIA A QUE FOI \nAPLICADA, NA ORIGEM, A \nSISTEMÁTICA DA REPERCUSSÃO \nGERAL E DESPROVIDO QUANTO À \nQUESTÃO REMANESCENTE. \n\nDECISÃO: Trata-se de agravo n

In [34]:
pipe = make_pipeline(TfidfVectorizer(min_df=5, norm=None),
LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(masked_dataset, label_train_bal)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

C:\Users\hlovi\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\hlovi\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\hlovi\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\hlovi\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Best cross-validation score: 0.96


C:\Users\hlovi\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
# transform the training dataset
X_train = vectorizer.transform(masked_dataset)
# find maximum value for each of the features over the dataset
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# get feature names
feature_names = np.array(vectorizer.get_feature_names())

In [49]:
for bp in range(len(grid.best_estimator_.named_steps["logisticregression"].coef_)):
    test_coef = grid.best_estimator_.named_steps["logisticregression"].coef_[bp]
    idx = np.argpartition(test_coef, -40)[-40:]
    indices = idx[np.argsort((-test_coef)[idx])]
    
    most_important_feature = feature_names[indices]
    print(bp, '---->', most_important_feature)

0 ----> ['reserva' 'adc' 'plenário' 'inconstitucionalidade' 'cláusula' 'região'
 'regional' 'trabalho' 'fracionário' 'subsidiária' 'apreciarei'
 'trabalhistas' 'município' 'órgão' 'normativo' 'norma' 'afastou' 'sem'
 'tribunal' 'domingues' 'bernardo' 'incompatibilidade' 'ltda' 'lei'
 'causa' 'afastado' 'responsabilidade' 'cf' 'intdo' 'ofensa'
 'declaratória' 'afasta' 'stj' 'declare' 'incidência' 'dê'
 'constitucionalidade' 'ribeirão' 'declarar' 'assembleia']
1 ----> ['algemas' 'júri' 'uso' 'audiência' 'prisão' 'vara' 'comarca' 'nulidade'
 'requisitório' 'juíza' 'algemado' 'manuel' 'aranalde' 'circunscrição'
 'solicitem' 'da' 'venceslau' 'lojistas' 'idelos' 'despacho' 'leme'
 'marinho' 'transgressão' 'arquive' 'carneiro' 'criminal' 'shopping'
 'juiz' 'luciana' 'avulsa' 'se' 'ministério' 'damacena' 'crime'
 'requerido' 'apresentou' 'geovani' 'feijó' 'flagrante' 'petição']
2 ----> ['acesso' 'criminal' 'inquérito' 'polícia' 'informações' 'autos' 'negado'
 'delegado' 'defesa' 'se' 'despacho

In [ ]:
  if bp_label[index] == '[3]':
        shutil.move(string, 'allbp/3')
    elif bp_label[index] == '[33]':
        shutil.move(string, 'allbp/33')
    elif bp_label[index] == '[37]':
        shutil.move(string, 'allbp/37')
    elif bp_label[index] == '[4]':
        shutil.move(string, 'allbp/4')
    elif bp_label[index] == '[14]':
        shutil.move(string, 'allbp/14')
    elif bp_label[index] == '[17]':
        shutil.move(string, 'allbp/17')
    elif bp_label[index] == '[26]':
        shutil.move(string, 'allbp/26')
    elif bp_label[index] == '[20]':
        shutil.move(string, 'allbp/20')
    elif bp_label[index] == '[10]':
        shutil.move(string, 'allbp/10')
    elif bp_label[index] == '[11]':
        shutil.move(string, 'allbp/11')